This is a re-implementation of the original transformers. This is for my general practice, for more of a tutorial structure. Consider going through my previous implementation.  

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [10]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, d_model):
        super().__init__()
        assert d_model%num_heads == 0

        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_model//num_heads

        self.W_Q = nn.Linear(self.d_model, self.d_model)
        self.W_K = nn.Linear(self.d_model, self.d_model)
        self.W_V = nn.Linear(self.d_model, self.d_model)
        self.W_O = nn.Linear(self.d_model, self.d_model)


    @staticmethod
    def scaled_dot_product_attention(Q,K,V, mask):
        """
        Args:
            query: (batch_size, num_heads, seq_len_q, d_q)
            key: (batch_size, num_heads, seq_len_k, d_k)
            value: (batch_size, num_heads, seq_len_v, d_v)
            mask: Optional mask to prevent attention to certain positions
        """
        assert Q.shape[-1] == K.shape[-1] #query and key dimension should be equal

        attention_score  = (torch.matmul(Q,K.transpose(-2,-1)))/torch.sqrt(torch.tensor(K.shape[-1]))
    
        if mask:
            upper_mask = torch.tril(torch.ones(attention_score.shape[-2], attention_score.shape[-1]))
            upper_mask.masked_fill_(upper_mask==0, float('-inf'))
            attention_score = attention_score + upper_mask

        attention_weights  = F.softmax(attention_score, dim=-1)
        assert attention_weights.shape == (Q.shape[0], Q.shape[1], Q.shape[2], K.shape[2])

        Z = torch.einsum('bhqk,bhkd -> bhqd', attention_weights, V)
        
        return Z
    
    def forward(self, input_matrix):
        batch_size, seq_len = input_matrix.shape[0], input_matrix.shape[1]

        self.Q = self.W_Q(input_matrix).reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        self.K = self.W_K(input_matrix).reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        self.V = self.W_V(input_matrix).reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)

        attention_score = self.scaled_dot_product_attention(self.Q, self.K, self.V, 0).transpose(1,2).reshape(batch_size, seq_len,self.d_model)
        return self.W_O(attention_score)

In [11]:
def test_attention():
    # Small test case
    batch_size, num_heads, seq_len, d_k = 2, 4, 6, 8
    d_model = num_heads * d_k  # 32
    
    print("=== Testing Scaled Dot Product Attention ===")
    
    # Create sample tensors for scaled_dot_product_attention
    Q = torch.randn(batch_size, num_heads, seq_len, d_k)
    K = torch.randn(batch_size, num_heads, seq_len, d_k)
    V = torch.randn(batch_size, num_heads, seq_len, d_k)
    
    # Test without mask
    output = MultiHeadAttention.scaled_dot_product_attention(Q, K, V, mask=False)
    print(f"SDPA Output shape: {output.shape}")
    print(f"SDPA Expected: {(batch_size, num_heads, seq_len, d_k)}")
    
    # Test with mask
    output_masked = MultiHeadAttention.scaled_dot_product_attention(Q, K, V, mask=True)
    print(f"SDPA Masked output shape: {output_masked.shape}")
    
    print("✅ SDPA tests passed!")
    
    print("\n=== Testing Full MultiHeadAttention ===")
    
    # Create MultiHeadAttention module
    mha = MultiHeadAttention(num_heads=num_heads, d_model=d_model)
    
    # Create input tensor (batch_size, seq_len, d_model)
    input_tensor = torch.randn(batch_size, seq_len, d_model)
    print(f"Input shape: {input_tensor.shape}")
    
    # Test forward pass
    try:
        mha_output = mha.forward(input_tensor)
        print(f"MHA Output shape: {mha_output.shape}")
        print(f"MHA Expected: {(batch_size, seq_len, d_model)}")
        
        # Check if output has reasonable values (not NaN or inf)
        if torch.isnan(mha_output).any():
            print("❌ Output contains NaN values!")
        elif torch.isinf(mha_output).any():
            print("❌ Output contains infinite values!")
        else:
            print("✅ Output values look reasonable!")
            
        print("✅ Full MHA test passed!")
        
    except Exception as e:
        print(f"❌ MHA test failed with error: {e}")
        print("Check your forward() method implementation")
    
    print("\n=== Testing with different input sizes ===")
    
    # Test with different sequence length
    seq_len_2 = 10
    input_tensor_2 = torch.randn(batch_size, seq_len_2, d_model)
    
    try:
        mha_output_2 = mha.forward(input_tensor_2)
        print(f"Different seq_len input: {input_tensor_2.shape}")
        print(f"Different seq_len output: {mha_output_2.shape}")
        print("✅ Variable sequence length test passed!")
    except Exception as e:
        print(f"❌ Variable sequence length test failed: {e}")

# Run the test
test_attention()

=== Testing Scaled Dot Product Attention ===
SDPA Output shape: torch.Size([2, 4, 6, 8])
SDPA Expected: (2, 4, 6, 8)
SDPA Masked output shape: torch.Size([2, 4, 6, 8])
✅ SDPA tests passed!

=== Testing Full MultiHeadAttention ===
Input shape: torch.Size([2, 6, 32])
MHA Output shape: torch.Size([2, 6, 32])
MHA Expected: (2, 6, 32)
✅ Output values look reasonable!
✅ Full MHA test passed!

=== Testing with different input sizes ===
Different seq_len input: torch.Size([2, 10, 32])
Different seq_len output: torch.Size([2, 10, 32])
✅ Variable sequence length test passed!


In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self):
        super().__init__()
        pass

    def forward(self, input_matrix):
        pass

In [ ]:
class Encoder(nn.Module):

    def __init__(self):
        super().__init__()

In [ ]:
class DecoderLayer(nn.Module):

    def __init__(self):
        super().__init__()
        pass

    def forward(self, input_matrix):
        pass

In [ ]:
class Decoder(nn.Module):

    def __init__(self):
        super().__init__()
        pass

    def forward(self, input_matrix):
        pass

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, seq_len):
        super().__init__()
        
        self.pe = torch.ones([seq_len,d_model])
        self.pos = torch.arange(seq_len).unsqueeze(1)
        self.dim = torch.arange(d_model).unsqueeze(0)

        # self.pe[:, ::2] = torch.sin(self.pos/10000**(2*self.dim[:d_model//2]/d_model)) # This slices about row, not column, So I still have the full tensor
        # self.pe[:, 1::2] = torch.cos(self.pos/10000**(2*self.dim[:d_model//2]/d_model))
        
        self.pe[:, ::2] = torch.sin(self.pos/10000**(2*self.dim[:, :d_model//2]/d_model)) # This slices it in half
        self.pe[:, 1::2] = torch.cos(self.pos/10000**(2*self.dim[:, :d_model//2]/d_model))

    def forward(self, input_matrix):
        return input_matrix + self.pe

In [ ]:
# test Positional Encoding 

#
def pe_matrix(d_model, seq_len):

    pe = torch.ones([seq_len,d_model])
    pos = torch.arange(seq_len).unsqueeze(1)
    dim = torch.arange(d_model).unsqueeze(0)

    pe[:, ::2] = torch.sin(pos/10000**(2*dim[:, :d_model//2]/d_model))
    pe[:, 1::2] = torch.cos(pos/10000**(2*dim[:, :d_model//2]/d_model))

    return pe

# Clearer approach
# def pe_matrix(d_model, seq_len):
#     pe = torch.zeros([seq_len, d_model])  
#     pos = torch.arange(seq_len).unsqueeze(1)
#     dim = torch.arange(d_model//2).unsqueeze(0)  
    
    
#     angles = pos / 10000**(2*dim/d_model)
    
#     pe[:, ::2] = torch.sin(angles)
#     pe[:, 1::2] = torch.cos(angles)
    
#     return pe

def test_pe_matrix(d_model, seq_len):
    print(f"Testing PE with d_model={d_model}, seq_len={seq_len}")
    
    # Generate PE matrix
    pe = pe_matrix(d_model, seq_len)
    
    # Test 1: Shape check
    print(f"Shape: {pe.shape} (expected: ({seq_len}, {d_model}))")
    assert pe.shape == (seq_len, d_model), f"Shape mismatch!"
    
    # Test 2: Print the matrix to visually inspect
    print("PE Matrix:")
    print(pe)
    
    # Test 3: Manual verification for position 0, dimensions 0 and 1
    pos_0 = 0
    expected_dim_0 = torch.sin(torch.tensor(pos_0 / 10000**(2*0/d_model)))  # i=0, so 2i=0
    expected_dim_1 = torch.cos(torch.tensor(pos_0 / 10000**(2*0/d_model)))  # same i=0
    
    print(f"\nManual check for position 0:")
    print(f"PE[0,0] = {pe[0,0]:.6f}, expected = {expected_dim_0:.6f}")
    print(f"PE[0,1] = {pe[0,1]:.6f}, expected = {expected_dim_1:.6f}")
    
    # Test 4: Check that values are different across positions
    if seq_len > 1:
        print(f"\nDifferent positions check:")
        print(f"PE[0,0] = {pe[0,0]:.6f}")
        print(f"PE[1,0] = {pe[1,0]:.6f}")
        print(f"Different? {not torch.allclose(pe[0,0], pe[1,0])}")
    
    print("✓ All tests passed!")

# Run the test
test_pe_matrix(4, 3)


Testing PE with d_model=4, seq_len=3
Shape: torch.Size([3, 4]) (expected: (3, 4))
PE Matrix:
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998]])

Manual check for position 0:
PE[0,0] = 0.000000, expected = 0.000000
PE[0,1] = 1.000000, expected = 1.000000

Different positions check:
PE[0,0] = 0.000000
PE[1,0] = 0.841471
Different? True
✓ All tests passed!


In [12]:
class FeedForwardNetwork(nn.Module):

    def __init__(self,d_model,d_ff,dropout=0.1):
        super().__init__()
        self.ff_model = nn.Sequential(
            nn.Linear(d_model,d_ff), 
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )

    def forward(self,input_matrix):
        return self.ff_model(input_matrix)

In [ ]:
class Transformer(nn.Module):

    def __init__(self):
        super().__init__()
        pass

    def forward(self, input_matrix):
        pass

In [ ]:
def train():
    pass